In [1]:
import os, math, random, re
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForCausalLM

# Reproducibility
SEED = 4738
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Device / dtype
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = (
    torch.bfloat16
    if DEVICE == "cuda" and torch.cuda.is_bf16_supported()
    else torch.float16
    if DEVICE == "cuda"
    else torch.float32
)
print("DEVICE:", DEVICE, "DTYPE:", DTYPE)

DEVICE: cuda DTYPE: torch.bfloat16


In [2]:
MODEL_NAME = "google/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

model_kwargs = dict(
    torch_dtype=DTYPE,
    device_map=None,
)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, **model_kwargs).to(DEVICE)
model.eval()

# IMPORTANT: we never train model weights; disabling grads prevents dtype/graph reuse issues
for p in model.parameters():
    p.requires_grad_(False)

MODEL_DTYPE = next(model.parameters()).dtype   # should be torch.bfloat16 (or fp16 on some GPUs)
DEVICE = next(model.parameters()).device

embed_layer = model.get_input_embeddings()

print("Loaded:", MODEL_NAME)
print("attn_implementation:", getattr(model.config, "attn_implementation", None))
print("vocab_size:", model.config.vocab_size, "d_model:", model.config.hidden_size)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded: google/gemma-2-2b-it
attn_implementation: None
vocab_size: 256000 d_model: 2304


In [3]:
P_REF = "Talk only about cats."

SUFFIXES = [
  "Write exactly two sentences explaining a simple process.",
  "Output valid JSON with keys \"a\" and \"b\", values as integers.",
  "Ask one clarifying question before providing any answer.",
  "If unsure, say you are not sure and explain why.",
  "Respond in exactly three bullet points, no extra text.",
  "Write a haiku about uncertainty using exactly seventeen syllables.",
  "Provide a concise definition using no commas.",
  "Ask a clarifying question, then wait for confirmation.",
  "Output a single word in all caps.",
  "Explain a concept in a friendly, informal tone.",
  "If the task seems ambiguous, refuse politely and state the ambiguity.",
  "Produce valid JSON array with three strings.",
  "Answer in exactly two sentences, each under ten words.",
  "Ask one clarifying question before attempting a solution.",
  "Write a short analogy involving tools and organization.",
  "Respond using exactly five words.",
  "If you lack confidence, say so and give a brief reason.",
  "Create an acrostic spelling ORDER, one word per line.",
  "Output YAML with keys alpha and beta only.",
  "Explain a simple rule as if to a beginner.",
  "Ask a clarifying question instead of answering directly.",
  "Respond with exactly three numbered steps.",
  "If information is insufficient, state uncertainty clearly.",
  "Write a polite refusal to answer an unclear request.",
  "Generate a single sentence without using the letter e.",
  "Output JSON with a boolean field named \"ok\".",
  "Explain the difference between speed and accuracy briefly.",
  "Ask one clarifying question, no additional commentary.",
  "Respond in a neutral, formal tone.",
  "If unsure, explicitly say \"I don't know\" and why.",
  "Write exactly four short lines forming a poem.",
  "Provide an example using abstract placeholders only.",
  "Return a CSV header line with three columns.",
  "Explain a tradeoff in exactly fifteen words.",
  "Ask a clarifying question before giving details.",
  "Respond with a checklist of three items.",
  "If the request is vague, refuse and explain succinctly.",
  "Write a single paragraph under forty words.",
  "Output valid JSON with nested object depth two.",
  "Summarize a hypothetical scenario in one sentence.",
  "Explain a simple algorithm conceptually, no code.",
  "Ask one clarifying question, then stop.",
  "Respond using only lowercase letters.",
  "If unsure, explain what additional information is needed.",
  "Create a title in title case, five words.",
  "Provide a definition followed by one example.",
  "Output exactly three emojis, no text.",
  "Ask a clarifying question before proceeding.",
  "Respond with exactly two clauses separated by a semicolon.",
  "If you cannot answer confidently, say so briefly.",
  "Write a rhyming couplet about balance.",
  "Output JSON with keys \"status\" and \"reason\".",
  "Explain a planning approach in three sentences.",
  "Ask one clarifying question only.",
  "Respond without using pronouns.",
  "If uncertain, decline and request clarification.",
  "Generate a slogan of exactly seven words.",
  "Provide a pros and cons list, two each.",
  "Output valid XML with a single root element.",
  "Explain a concept using a metaphor.",
  "Ask a clarifying question prior to answering.",
  "Respond in exactly one sentence.",
  "If information is missing, state uncertainty.",
  "Write instructions using imperative verbs only.",
  "Output JSON where values are empty strings.",
  "Explain a mistake and how to avoid it.",
  "Ask one clarifying question before any explanation.",
  "Respond with exactly three short paragraphs.",
  "If unsure, explain limitations of the answer.",
  "Create a mnemonic using initial letters.",
  "Provide a minimal example, no explanation.",
  "Output a Markdown table with two columns, two rows.",
  "Explain a decision-making principle briefly.",
  "Ask a clarifying question and do not answer.",
  "Respond using exactly ten words.",
  "If the prompt is ambiguous, politely refuse.",
  "Write a neutral summary in two sentences.",
  "Output JSON with an array of two numbers.",
  "Explain cause and effect succinctly.",
  "Ask one clarifying question first.",
  "Respond without punctuation marks.",
  "If unsure, say so and suggest next steps.",
  "Create a short checklist with checkboxes.",
  "Provide a one-line definition.",
  "Output exactly three hashtags.",
  "Explain a workflow step-by-step, briefly.",
  "Ask a clarifying question before solving.",
  "Respond in a calm, professional tone.",
  "If information is insufficient, decline respectfully.",
  "Write a closing sentence for a report.",
  "Output valid JSON with key \"result\" only."
]

max_n_suffixes = 10
SUFFIXES = SUFFIXES[:max_n_suffixes]

# Deterministic decoding so "same output" is meaningful
GEN_KW = dict(
    do_sample=False,
    num_beams=1,
    max_new_tokens=60,   # smaller while iterating; increase later
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

# Token-level matching metric for the *first K completion tokens*
MATCH_FIRST_K = 50

# Weight early completion tokens more heavily in CE loss (stabilizes exact-match)
EARLY_K = 32
EARLY_WEIGHT = 3.0

# Steering length: currently match the reference length exactly
P_REF_IDS = tokenizer(P_REF, add_special_tokens=False).input_ids
STEER_LEN = len(P_REF_IDS)
print("Reference tokens:", len(P_REF_IDS), P_REF_IDS[:10], "...")
print("STEER_LEN:", STEER_LEN)

# Chat-template scaffolding (recommended for instruction-tuned Gemma)
USE_CHAT_TEMPLATE = True
SEP_TEXT = "\n\n"

def _find_subseq(hay: List[int], needle: List[int]) -> int:
    """Return start index of needle in hay, or -1."""
    if not needle:
        return -1
    for i in range(0, len(hay) - len(needle) + 1):
        if hay[i : i + len(needle)] == needle:
            return i
    return -1

@dataclass
class ChatScaffold:
    # For generation (user message -> assistant generation prompt)
    gen_pre: List[int]
    gen_post: List[int]
    # For teacher forcing (user message + assistant message content)
    tf_pre: List[int]
    tf_between: List[int]
    tf_post: List[int]

def build_chat_scaffold(tok: AutoTokenizer) -> ChatScaffold:
    """
    Build token-level scaffolds using tokenizer.apply_chat_template and placeholders,
    so we can splice token IDs reliably without re-tokenizing whole prompts.
    """
    if not getattr(tok, "apply_chat_template", None):
        raise RuntimeError("Tokenizer has no chat template; set USE_CHAT_TEMPLATE=False or use a compatible tokenizer.")

    user_ph = "<<<USER_CONTENT_PLACEHOLDER>>>"
    asst_ph = "<<<ASSISTANT_CONTENT_PLACEHOLDER>>>"

    user_ph_ids = tok(user_ph, add_special_tokens=False).input_ids
    asst_ph_ids = tok(asst_ph, add_special_tokens=False).input_ids

    # Generation scaffold: user(content) then assistant prompt
    gen_ids = tok.apply_chat_template(
        [{"role": "user", "content": user_ph}],
        tokenize=True,
        add_generation_prompt=True,
    )
    if isinstance(gen_ids, torch.Tensor):
        gen_ids = gen_ids.tolist()
    u0 = _find_subseq(gen_ids, user_ph_ids)
    if u0 < 0:
        raise RuntimeError("Couldn't locate user placeholder tokens inside gen chat template.")
    gen_pre = gen_ids[:u0]
    gen_post = gen_ids[u0 + len(user_ph_ids) :]

    # Teacher-forcing scaffold: user(content) + assistant(content)
    tf_ids = tok.apply_chat_template(
        [{"role": "user", "content": user_ph}, {"role": "assistant", "content": asst_ph}],
        tokenize=True,
        add_generation_prompt=False,
    )
    if isinstance(tf_ids, torch.Tensor):
        tf_ids = tf_ids.tolist()
    u1 = _find_subseq(tf_ids, user_ph_ids)
    a1 = _find_subseq(tf_ids, asst_ph_ids)
    if u1 < 0 or a1 < 0 or a1 <= u1:
        raise RuntimeError("Couldn't locate placeholders inside teacher-forcing chat template.")
    tf_pre = tf_ids[:u1]
    tf_between = tf_ids[u1 + len(user_ph_ids) : a1]
    tf_post = tf_ids[a1 + len(asst_ph_ids) :]

    return ChatScaffold(gen_pre=gen_pre, gen_post=gen_post, tf_pre=tf_pre, tf_between=tf_between, tf_post=tf_post)

if USE_CHAT_TEMPLATE:
    CHAT = build_chat_scaffold(tokenizer)
    SEP_IDS = tokenizer(SEP_TEXT, add_special_tokens=False).input_ids
else:
    CHAT = None
    SEP_IDS = tokenizer(SEP_TEXT, add_special_tokens=False).input_ids


Reference tokens: 5 [27586, 1297, 1105, 19493, 235265] ...
STEER_LEN: 5


In [4]:
@torch.no_grad()
def greedy_generate_ids(prefix_ids: List[int], suffix: str, **gen_kw) -> str:
    """
    Greedy-generate assistant completion given a discrete prefix token list + suffix text.
    Uses chat template scaffolding if enabled.
    Returns completion text (decoded generated tokens after the prompt).
    """
    sfx_ids = tokenizer(suffix, add_special_tokens=False).input_ids

    if USE_CHAT_TEMPLATE:
        prompt_ids = CHAT.gen_pre + prefix_ids + SEP_IDS + sfx_ids + CHAT.gen_post
    else:
        prompt_ids = prefix_ids + SEP_IDS + sfx_ids

    input_ids = torch.tensor([prompt_ids], dtype=torch.long, device=DEVICE)
    attn = torch.ones_like(input_ids, dtype=torch.long)

    out = model.generate(input_ids=input_ids, attention_mask=attn, **gen_kw)
    gen_ids = out[0].tolist()[len(prompt_ids):]
    return tokenizer.decode(gen_ids, skip_special_tokens=True)

@torch.no_grad()
def greedy_generate_text(prefix_text: str, suffix: str, **gen_kw) -> str:
    prefix_ids = tokenizer(prefix_text, add_special_tokens=False).input_ids
    return greedy_generate_ids(prefix_ids, suffix, **gen_kw)

# Build deterministic reference outputs
Y_REF: Dict[str, str] = {}
for sfx in SUFFIXES:
    Y_REF[sfx] = greedy_generate_text(P_REF, sfx, **GEN_KW)

for sfx in SUFFIXES[:2]:
    print("SUFFIX:", sfx)
    print("Y_REF:", Y_REF[sfx][:220], "...\n")

SUFFIX: Write exactly two sentences explaining a simple process.
Y_REF: Cats groom themselves by licking their fur, removing dirt and loose hairs.  This process helps keep their coat healthy and shiny. 
 ...

SUFFIX: Output valid JSON with keys "a" and "b", values as integers.
Y_REF: ```json
{
  "a": 3,
  "b": 17
}
``` ...



In [5]:
@dataclass
class Example:
    suffix: str
    ref_completion: str
    suffix_ids: torch.Tensor        # [Ts]
    completion_ids: torch.Tensor    # [Tc]

def build_example(suffix: str, completion: str) -> Example:
    sfx_ids = torch.tensor(
        tokenizer(suffix, add_special_tokens=False).input_ids,
        dtype=torch.long,
    )
    comp_ids = torch.tensor(
        tokenizer(completion, add_special_tokens=False).input_ids,
        dtype=torch.long,
    )
    return Example(suffix=suffix, ref_completion=completion, suffix_ids=sfx_ids, completion_ids=comp_ids)

examples: List[Example] = [build_example(sfx, Y_REF[sfx]) for sfx in SUFFIXES]
max_len = max((ex.suffix_ids.numel() + ex.completion_ids.numel()) for ex in examples)
print("Num examples:", len(examples), "Max suffix+completion length:", max_len)

Num examples: 10 Max suffix+completion length: 72


In [6]:
def pad_1d(seqs: List[torch.Tensor], pad_value: int) -> torch.Tensor:
    max_len = max(x.numel() for x in seqs)
    out = torch.full((len(seqs), max_len), pad_value, dtype=seqs[0].dtype, device=seqs[0].device)
    for i, x in enumerate(seqs):
        out[i, : x.numel()] = x
    return out

def make_batch_with_prefix_ids(prefix_ids: torch.Tensor, batch: List[Example]) -> Dict[str, torch.Tensor]:
    """
    Build padded batch for teacher-forcing:
      input_ids = [chat_pre] + prefix_ids + [SEP] + suffix_ids + [chat_between] + completion_ids + [chat_post]
    labels are -100 everywhere except completion token positions.
    loss_weights are 0 on ignored positions, and (EARLY_WEIGHT or 1.0) on completion positions
    (heavier for first EARLY_K completion tokens).
    """
    device = DEVICE
    prefix_ids = prefix_ids.to(device)

    pre = torch.tensor(CHAT.tf_pre if USE_CHAT_TEMPLATE else [], dtype=torch.long, device=device)
    between = torch.tensor(CHAT.tf_between if USE_CHAT_TEMPLATE else [], dtype=torch.long, device=device)
    post = torch.tensor(CHAT.tf_post if USE_CHAT_TEMPLATE else [], dtype=torch.long, device=device)
    sep = torch.tensor(SEP_IDS, dtype=torch.long, device=device)

    input_ids_list, labels_list, attn_list, w_list = [], [], [], []

    for ex in batch:
        sfx = ex.suffix_ids.to(device)
        comp = ex.completion_ids.to(device)

        ids = torch.cat([pre, prefix_ids, sep, sfx, between, comp, post], dim=0)

        labels = torch.full_like(ids, -100)
        weights = torch.zeros_like(ids, dtype=torch.float)

        # completion starts after: pre + prefix + sep + suffix + between
        comp_start = pre.numel() + prefix_ids.numel() + sep.numel() + sfx.numel() + between.numel()
        labels[comp_start : comp_start + comp.numel()] = comp

        # early-token weighting
        for i in range(comp.numel()):
            w = EARLY_WEIGHT if i < EARLY_K else 1.0
            weights[comp_start + i] = w

        attn = torch.ones_like(ids, dtype=torch.long)

        input_ids_list.append(ids)
        labels_list.append(labels)
        attn_list.append(attn)
        w_list.append(weights)

    input_ids = pad_1d(input_ids_list, pad_value=tokenizer.pad_token_id)
    labels = pad_1d(labels_list, pad_value=-100)
    attention_mask = pad_1d(attn_list, pad_value=0)
    loss_weights = pad_1d(w_list, pad_value=0).to(torch.float)

    return {"input_ids": input_ids, "labels": labels, "attention_mask": attention_mask, "loss_weights": loss_weights}

In [7]:
def weighted_teacher_forced_ce(prefix_ids: torch.Tensor, batch: List[Example]) -> torch.Tensor:
    b = make_batch_with_prefix_ids(prefix_ids, batch)
    out = model(input_ids=b["input_ids"], attention_mask=b["attention_mask"])
    logits = out.logits[:, :-1, :].contiguous()
    labels = b["labels"][:, 1:].contiguous()
    weights = b["loss_weights"][:, 1:].contiguous()

    # per-token loss
    per = F.cross_entropy(
        logits.view(-1, logits.size(-1)),
        labels.view(-1),
        ignore_index=-100,
        reduction="none",
    ).view(labels.shape)

    mask = (labels != -100).float()
    denom = (weights * mask).sum().clamp_min(1.0)
    return (per * weights * mask).sum() / denom

@torch.no_grad()
def batch_ce(prefix_ids: List[int], batch: List[Example]) -> float:
    p = torch.tensor(prefix_ids, dtype=torch.long, device=DEVICE)
    return float(weighted_teacher_forced_ce(p, batch).item())

In [8]:
def levenshtein(a: List[int], b: List[int]) -> int:
    n, m = len(a), len(b)
    dp = list(range(m + 1))
    for i in range(1, n + 1):
        prev = dp[0]
        dp[0] = i
        for j in range(1, m + 1):
            cur = dp[j]
            cost = 0 if a[i - 1] == b[j - 1] else 1
            dp[j] = min(dp[j] + 1, dp[j - 1] + 1, prev + cost)
            prev = cur
    return dp[m]

@dataclass
class PromptConstraints:
    ref_ids: List[int]
    max_token_overlap_frac: float = 0.25
    min_edit_distance: int = 4
    forbid_ref_substring: bool = True
    require_printable: bool = True
    ascii_only: bool = True
    max_len_chars: int = 80
    # NEW: phrase-level bans
    forbid_phrases: Optional[List[str]] = None
    phrase_case_insensitive: bool = True

def constraints_ok(prefix_ids: List[int], cons: PromptConstraints) -> bool:
    ref_ids = cons.ref_ids

    # overlap
    if ref_ids:
        overlap = len(set(prefix_ids) & set(ref_ids))
        if overlap / max(1, len(set(ref_ids))) > cons.max_token_overlap_frac:
            return False

    # edit distance
    if levenshtein(prefix_ids, ref_ids) < cons.min_edit_distance:
        return False

    # decoded string once
    s = tokenizer.decode(prefix_ids, skip_special_tokens=True)

    # decoded substring (exact ref string)
    if cons.forbid_ref_substring and ref_ids:
        b = tokenizer.decode(ref_ids, skip_special_tokens=True)
        if b.strip() and (b.strip() in s):
            return False

    # NEW: phrase-level bans (handles “Talk only about cats...” etc)
    if cons.forbid_phrases:
        def _norm(text: str) -> str:
            if cons.phrase_case_insensitive:
                text = text.lower()
            text = re.sub(r"\s+", " ", text)
            return text.strip()

        s_norm = _norm(s)
        for ph in cons.forbid_phrases:
            if not ph:
                continue
            if _norm(ph) in s_norm:
                return False

    # readability
    if not s.strip():
        return False
    if cons.require_printable and any(not ch.isprintable() for ch in s):
        return False
    if cons.ascii_only and any(ord(ch) > 127 for ch in s):
        return False
    if len(s) > cons.max_len_chars:
        return False

    return True

constraints = PromptConstraints(
    ref_ids=P_REF_IDS,
    min_edit_distance=max(4, len(P_REF_IDS)),
    # Ban the teacher prompt text itself (you can add more phrases here)
    forbid_phrases=[P_REF, *P_REF.split(" ")],
)

constraints

PromptConstraints(ref_ids=[27586, 1297, 1105, 19493, 235265], max_token_overlap_frac=0.25, min_edit_distance=5, forbid_ref_substring=True, require_printable=True, ascii_only=True, max_len_chars=80, forbid_phrases=['Talk only about cats.', 'Talk', 'only', 'about', 'cats.'], phrase_case_insensitive=True)

In [9]:
def build_banned_token_ids(phrases: List[str]) -> set:
    """
    Build a set of token IDs that should never be used in the steering prefix,
    by tokenizing several simple variants of each phrase.
    """
    variants = []
    for p in phrases:
        if not p:
            continue
        variants.extend([
            p,
            " " + p,
            "\n" + p,
            "\n\n" + p,
            p.lower(),
            p.upper(),
        ])
    banned = set()
    for v in variants:
        ids = tokenizer(v, add_special_tokens=False).input_ids
        banned.update(ids)
    return banned

In [10]:
# %%
class SoftSteeringPrompt(nn.Module):
    """
    Soft steering prompt parameterized as per-position logits over the vocab.

    Forward:
      - Applies softmax over logits to get a token distribution per position.
      - Multiplies by the (frozen) embedding matrix to get a convex combination
        of *valid token embeddings* at each position.
    """
    def __init__(
        self,
        prompt_len: int,
        vocab_size: int,
        init_from_ref_ids: Optional[List[int]] = None,
        init_std: float = 0.5,
        *,
        device=None,
    ):
        super().__init__()
        if device is None:
            device = DEVICE

        self.prompt_len = prompt_len
        self.vocab_size = vocab_size

        # We keep logits in float32 for numerical stability.
        self.logits = nn.Parameter(
            torch.zeros(prompt_len, vocab_size, device=device, dtype=torch.float32)
        )

        # Initialize around the reference tokens if provided, otherwise random.
        with torch.no_grad():
            if init_from_ref_ids:
                ids = list(init_from_ref_ids)
                if len(ids) < prompt_len:
                    reps = math.ceil(prompt_len / len(ids))
                    ids = (ids * reps)[:prompt_len]
                else:
                    ids = ids[:prompt_len]

                # Start with small noise, then give a positive bias to the ref token at each pos.
                self.logits.data.normal_(mean=0.0, std=init_std)
                for i, tok in enumerate(ids):
                    if 0 <= tok < vocab_size:
                        self.logits.data[i, tok] += 3.0
            else:
                self.logits.data.normal_(mean=0.0, std=init_std)

    def forward(self) -> torch.Tensor:
        """
        Returns soft prompt embeddings of shape [L, d_model] in MODEL_DTYPE,
        each row a convex combination of token embeddings.
        """
        # [L, V] float32
        logits = self.logits
        probs = F.softmax(logits, dim=-1)

        # Embedding matrix: [V, d]; cast to float32 for matmul, then back to MODEL_DTYPE
        E = embed_layer.weight.to(torch.float32)
        emb = probs @ E  # [L, d] float32

        return emb.to(dtype=MODEL_DTYPE, device=DEVICE)


def train_soft_prompt(
    prompt_len: int,
    batch_examples: List[Example],
    steps: int = 200,
    lr: float = 2e-2,
    batch_size: int = 4,
    init_from_ref: Optional[List[int]] = None,
    repulsion_weight: float = 0.03,
) -> SoftSteeringPrompt:
    """
    Optimize a *token-mixture* soft prompt so that, under suffix prompts, the model
    assigns high likelihood to the reference completions (weighted teacher-forced CE).

    The soft embeddings at each position are convex combinations of token embeddings,
    which keeps them close to valid token embeddings (reduces soft→discrete mismatch).
    """
    vocab_size = embed_layer.weight.shape[0]
    sp = SoftSteeringPrompt(
        prompt_len=prompt_len,
        vocab_size=vocab_size,
        init_from_ref_ids=init_from_ref,
        device=DEVICE,
    )

    # We optimize logits (float32) only.
    opt = torch.optim.AdamW([sp.logits], lr=lr, betas=(0.9, 0.98), weight_decay=0.0)

    # fixed tensors for chat scaffold
    pre = torch.tensor(CHAT.tf_pre if USE_CHAT_TEMPLATE else [], dtype=torch.long, device=DEVICE)
    between = torch.tensor(CHAT.tf_between if USE_CHAT_TEMPLATE else [], dtype=torch.long, device=DEVICE)
    post = torch.tensor(CHAT.tf_post if USE_CHAT_TEMPLATE else [], dtype=torch.long, device=DEVICE)
    sep = torch.tensor(SEP_IDS, dtype=torch.long, device=DEVICE)

    # Cache scaffold embeddings, DETACHED so they don't keep a graph across steps
    with torch.no_grad():
        pre_emb = embed_layer(pre).unsqueeze(0) if pre.numel() else None
        between_emb = embed_layer(between).unsqueeze(0) if between.numel() else None
        post_emb = embed_layer(post).unsqueeze(0) if post.numel() else None
        sep_emb = embed_layer(sep).unsqueeze(0)

        ref_ids_for_mean = tokenizer(P_REF, add_special_tokens=False).input_ids
        ref_mean = embed_layer(torch.tensor(ref_ids_for_mean, device=DEVICE)).mean(dim=0, keepdim=True)  # [1,d]

    ref_mean = ref_mean.to(dtype=MODEL_DTYPE)

    for step in range(steps):
        batch = random.sample(batch_examples, k=min(batch_size, len(batch_examples)))

        emb_seqs = []
        labels_seqs = []
        attn_seqs = []
        w_seqs = []

        for ex in batch:
            sfx = ex.suffix_ids.to(DEVICE)
            comp = ex.completion_ids.to(DEVICE)

            ids = torch.cat([sep, sfx, between, comp, post], dim=0)
            with torch.no_grad():
                emb = embed_layer(ids).unsqueeze(0)  # [1,T,d] detached by no_grad
            emb = emb.to(dtype=MODEL_DTYPE)

            full_len = (pre.numel() + prompt_len + ids.numel())
            labels = torch.full((full_len,), -100, dtype=torch.long, device=DEVICE)
            weights = torch.zeros((full_len,), dtype=torch.float, device=DEVICE)

            comp_start = pre.numel() + prompt_len + sep.numel() + sfx.numel() + between.numel()
            labels[comp_start : comp_start + comp.numel()] = comp
            for i in range(comp.numel()):
                weights[comp_start + i] = (EARLY_WEIGHT if i < EARLY_K else 1.0)

            attn = torch.ones((full_len,), dtype=torch.long, device=DEVICE)

            emb_seqs.append(emb)
            labels_seqs.append(labels)
            attn_seqs.append(attn)
            w_seqs.append(weights)

        max_full = max(x.numel() for x in labels_seqs)
        B = len(batch)
        d = model.config.hidden_size

        # IMPORTANT: ensure inputs_embeds matches model dtype
        inputs_embeds = torch.zeros((B, max_full, d), dtype=MODEL_DTYPE, device=DEVICE)
        labels = torch.full((B, max_full), -100, dtype=torch.long, device=DEVICE)
        attn = torch.zeros((B, max_full), dtype=torch.long, device=DEVICE)
        weights = torch.zeros((B, max_full), dtype=torch.float, device=DEVICE)

        # soft prompt embeddings from logits (convex combination of token embeddings)
        soft = sp().unsqueeze(0)  # [1,L,d] in MODEL_DTYPE
        for i in range(B):
            L_i = labels_seqs[i].numel()

            off = 0
            if pre_emb is not None:
                inputs_embeds[i, off : off + pre_emb.shape[1], :] = pre_emb[0].to(dtype=MODEL_DTYPE)
                off += pre_emb.shape[1]

            inputs_embeds[i, off : off + prompt_len, :] = soft[0]
            off += prompt_len

            rest = emb_seqs[i]
            inputs_embeds[i, off : off + rest.shape[1], :] = rest[0]
            off += rest.shape[1]

            labels[i, :L_i] = labels_seqs[i]
            attn[i, :L_i] = attn_seqs[i]
            weights[i, :L_i] = w_seqs[i]

        # Defensive cast before forward
        inputs_embeds = inputs_embeds.to(dtype=MODEL_DTYPE, device=DEVICE)

        out = model(inputs_embeds=inputs_embeds, attention_mask=attn)
        logits = out.logits[:, :-1, :]
        lab = labels[:, 1:]
        w = weights[:, 1:]
        per = F.cross_entropy(
            logits.reshape(-1, logits.size(-1)),
            lab.reshape(-1),
            ignore_index=-100,
            reduction="none",
        ).view_as(lab)

        mask = (lab != -100).float()
        ce = (per * w * mask).sum() / (w * mask).sum().clamp_min(1.0)

        # repulsion from reference mean (keep small; constraints do the real "different")
        soft_mean = soft.mean(dim=1)  # [1,d]
        rep = F.cosine_similarity(soft_mean, ref_mean, dim=-1).mean()
        loss = ce + repulsion_weight * rep

        opt.zero_grad(set_to_none=True)
        loss.backward()
        opt.step()

        if step % 25 == 0 or step == steps - 1:
            print(f"[soft step {step:4d}] loss={loss.item():.4f}  ce={ce.item():.7f}")

    return sp


In [11]:
# %%
# %%
@torch.no_grad()
def project_soft_to_tokens(
    logits: torch.Tensor,
    banned_token_ids: Optional[set] = None,
) -> List[int]:
    """
    Project per-position logits over the vocab to a discrete token ID per position.

    logits: [L, V] float tensor (e.g. from SoftSteeringPrompt.logits)
    Returns: List[int] of length L, best token per position, skipping banned_token_ids.
    """
    if banned_token_ids is None:
        banned_token_ids = set()

    L, V = logits.shape
    out_ids: List[int] = []

    for i in range(L):
        row = logits[i].clone()
        # Mask out banned IDs for this position
        for t in banned_token_ids:
            if 0 <= t < V:
                row[t] = float("-inf")
        best_id = int(torch.argmax(row).item())
        out_ids.append(best_id)

    return out_ids


In [12]:
@torch.no_grad()
def eval_discrete_prefix(prefix_ids: List[int], suffixes: List[str], y_ref: Dict[str, str]) -> Dict[str, float]:
    batch = [build_example(s, y_ref[s]) for s in suffixes]
    p = torch.tensor(prefix_ids, dtype=torch.long, device=DEVICE)
    ce = float(weighted_teacher_forced_ce(p, batch).item())

    exact = 0
    exact_k = 0
    tok_match_sum = 0
    tok_total = 0
    tok_match_k_sum = 0
    tok_total_k = 0

    for s in suffixes:
        cand = greedy_generate_ids(prefix_ids, s, **GEN_KW)
        ref = y_ref[s]
        if cand == ref:
            exact += 1

        c_ids = tokenizer(cand, add_special_tokens=False).input_ids
        r_ids = tokenizer(ref, add_special_tokens=False).input_ids

        L = min(len(c_ids), len(r_ids))
        tok_match_sum += sum(1 for i in range(L) if c_ids[i] == r_ids[i])
        tok_total += max(1, len(r_ids))

        K = min(MATCH_FIRST_K, len(r_ids), len(c_ids))
        tok_match_k_sum += sum(1 for i in range(K) if c_ids[i] == r_ids[i])
        tok_total_k += max(1, min(MATCH_FIRST_K, len(r_ids)))

        if c_ids[:K] == r_ids[:K] and K > 0:
            exact_k += 1

    n = max(1, len(suffixes))
    return {
        "ce_loss": ce,
        "exact_match_rate": exact / n,
        "exact_match_rate_firstk": exact_k / n,
        "token_match_rate": tok_match_sum / max(1, tok_total),
        "token_match_rate_firstk": tok_match_k_sum / max(1, tok_total_k),
    }

In [13]:
# %%
def compute_prefix_grads(prefix_ids: List[int], batch: List[Example]) -> torch.Tensor:
    """
    Compute average gradient w.r.t. prefix *position embeddings* for the current discrete prefix.
    Returns [L,d].
    """
    device = DEVICE
    p = torch.tensor(prefix_ids, dtype=torch.long, device=device)
    b = make_batch_with_prefix_ids(p, batch)

    # Embed full sequence and backprop through embeddings directly
    inputs_embeds = embed_layer(b["input_ids"]).detach().to(dtype=MODEL_DTYPE)
    inputs_embeds.requires_grad_(True)

    out = model(inputs_embeds=inputs_embeds, attention_mask=b["attention_mask"])
    logits = out.logits[:, :-1, :]
    labels = b["labels"][:, 1:]
    weights = b["loss_weights"][:, 1:]

    per = F.cross_entropy(
        logits.reshape(-1, logits.size(-1)),
        labels.reshape(-1),
        ignore_index=-100,
        reduction="none",
    ).view_as(labels)

    mask = (labels != -100).float()
    loss = (per * weights * mask).sum() / (weights * mask).sum().clamp_min(1.0)

    # IMPORTANT: clear any existing grads
    if inputs_embeds.grad is not None:
        inputs_embeds.grad.zero_()
    loss.backward()

    # Prefix starts after chat tf_pre
    pre_len = len(CHAT.tf_pre) if USE_CHAT_TEMPLATE else 0
    L = len(prefix_ids)
    g = inputs_embeds.grad[:, pre_len : pre_len + L, :].mean(dim=0)  # [L,d]
    return g.detach()

@torch.no_grad()
def hotflip_candidates(
    grad: torch.Tensor,                 # [d]
    current_id: int,
    top_k: int = 32,
    banned_token_ids: Optional[set] = None,
) -> List[int]:
    """
    HotFlip: choose token replacements that most reduce loss (approx) via grad · (e_new - e_cur).
    """
    if banned_token_ids is None:
        banned_token_ids = set()

    w = embed_layer.weight.detach()  # [V,d]
    # use float32 scoring for stability
    g = grad.float()
    w_f = w.float()

    scores = torch.mv(w_f, g)  # [V]
    scores[current_id] = float("inf")
    for t in banned_token_ids:
        if 0 <= t < scores.numel():
            scores[t] = float("inf")

    _, idx = torch.topk(scores, k=top_k, largest=False)
    return idx.tolist()


In [14]:
# %%
def refine_discrete_prefix(
    init_prefix_ids: List[int],
    batch_examples: List[Example],
    cons: PromptConstraints,
    passes: int = 6,
    hotflip_top_k: int = 64,
    eval_top_k_per_pos: int = 12,
    banned_token_ids: Optional[set] = None,
) -> List[int]:
    if banned_token_ids is None:
        banned_token_ids = set()

    prefix = init_prefix_ids[:]
    best = batch_ce(prefix, batch_examples)
    print("Init CE:", best)

    for p_i in range(passes):
        improved = False
        grads = compute_prefix_grads(prefix, batch_examples)  # [L,d]

        for pos in range(len(prefix)):
            cur_id = prefix[pos]
            cand_ids = hotflip_candidates(grads[pos], cur_id, top_k=hotflip_top_k, banned_token_ids=banned_token_ids)

            shortlist = cand_ids[:eval_top_k_per_pos]
            local_best = best
            local_best_tok = cur_id

            for tok in shortlist:
                trial = prefix[:]
                trial[pos] = tok
                if not constraints_ok(trial, cons):
                    continue
                loss = batch_ce(trial, batch_examples)
                if loss < local_best:
                    local_best = loss
                    local_best_tok = tok

            if local_best_tok != cur_id:
                prefix[pos] = local_best_tok
                best = local_best
                improved = True
                print(f"[pass {p_i+1}/{passes}] pos {pos:2d} -> loss {best:.7f}")

        if not improved:
            print(f"[pass {p_i+1}/{passes}] no improvement; stopping.")
            break

    return prefix


In [15]:
# %%
@dataclass
class Candidate:
    prefix_ids: List[int]
    prefix_text: str
    ce_loss: float
    exact_match_rate: float
    exact_match_rate_firstk: float
    token_match_rate: float
    token_match_rate_firstk: float

def generate_steering_prompt(
    p_ref: str,
    suffixes: List[str],
    steer_len: int,
    cons: PromptConstraints,
    soft_steps: int = 200,
    soft_lr: float = 2e-2,
    refine_passes: int = 6,
    n_restarts: int = 12,
) -> Candidate:
    # Build deterministic reference completions
    y_ref = {s: greedy_generate_text(p_ref, s, **GEN_KW) for s in suffixes}
    exs = [build_example(s, y_ref[s]) for s in suffixes]

    ref_ids = tokenizer(p_ref, add_special_tokens=False).input_ids

    # Build a richer banned-token set from the teacher prompt text and cat-ish tokens
    banned_token_ids = build_banned_token_ids([p_ref, "cats", "cat", "talk"])

    best_cand: Optional[Candidate] = None

    for r in range(n_restarts):
        torch.manual_seed(SEED + 1000 + r)
        random.seed(SEED + 1000 + r)

        sp = train_soft_prompt(
            prompt_len=steer_len,
            batch_examples=exs,
            steps=soft_steps,
            lr=soft_lr,
            batch_size=min(4, len(exs)),
            init_from_ref=ref_ids,
            repulsion_weight=0.03,
        )

        # Project soft (logits over tokens) -> discrete while avoiding banned IDs
        proj_ids = project_soft_to_tokens(sp.logits.detach(), banned_token_ids=banned_token_ids)
        if not constraints_ok(proj_ids, cons):
            # Small perturbation in *logit space* and reproject
            noisy_logits = sp.logits.detach() + 0.02 * torch.randn_like(sp.logits.detach())
            proj_ids = project_soft_to_tokens(noisy_logits, banned_token_ids=banned_token_ids)

        refined = refine_discrete_prefix(
            init_prefix_ids=proj_ids,
            batch_examples=exs,
            cons=cons,
            passes=refine_passes,
            hotflip_top_k=64,
            eval_top_k_per_pos=18,
            banned_token_ids=banned_token_ids,
        )

        stats = eval_discrete_prefix(refined, suffixes, y_ref)
        cand = Candidate(
            prefix_ids=refined,
            prefix_text=tokenizer.decode(refined, skip_special_tokens=True),
            ce_loss=stats["ce_loss"],
            exact_match_rate=stats["exact_match_rate"],
            exact_match_rate_firstk=stats["exact_match_rate_firstk"],
            token_match_rate=stats["token_match_rate"],
            token_match_rate_firstk=stats["token_match_rate_firstk"],
        )

        if (best_cand is None) or (cand.exact_match_rate_firstk, -cand.ce_loss) > (best_cand.exact_match_rate_firstk, -best_cand.ce_loss):
            best_cand = cand

        print(
            f"[restart {r+1}/{n_restarts}] "
            f"CE={cand.ce_loss:.4f} "
            f"exactK={cand.exact_match_rate_firstk:.3f} "
            f"tokK={cand.token_match_rate_firstk:.3f}  "
            f"prompt={cand.prefix_text!r}"
        )

    assert best_cand is not None
    return best_cand

cand = generate_steering_prompt(
    p_ref=P_REF,
    suffixes=SUFFIXES,
    steer_len=STEER_LEN,
    cons=constraints,
    soft_steps=400,
    soft_lr=2e-2,
    refine_passes=12,
    n_restarts=10,
)

print("\nREFERENCE PREFIX:", repr(P_REF))
print("CANDIDATE PREFIX:", repr(cand.prefix_text))
print(
    "Candidate metrics:",
    "CE:", cand.ce_loss,
    "Exact:", cand.exact_match_rate,
    "Exact@K:", cand.exact_match_rate_firstk,
    "Tok@K:", cand.token_match_rate_firstk,
)


[soft step    0] loss=1.0149  ce=1.0251418
[soft step   25] loss=1.2324  ce=1.2425153
[soft step   50] loss=1.3405  ce=1.3506116
[soft step   75] loss=0.9909  ce=1.0009747
[soft step  100] loss=1.1603  ce=1.1703049
[soft step  125] loss=0.8686  ce=0.8786584
[soft step  150] loss=0.7446  ce=0.7546949
[soft step  175] loss=0.7676  ce=0.7777804
[soft step  200] loss=0.6841  ce=0.6942280
[soft step  225] loss=0.5782  ce=0.5883546
[soft step  250] loss=0.5042  ce=0.5143426
[soft step  275] loss=0.4960  ce=0.5061175
[soft step  300] loss=0.5527  ce=0.5627846
[soft step  325] loss=0.5332  ce=0.5433112
[soft step  350] loss=0.4657  ce=0.4759127
[soft step  375] loss=0.4544  ce=0.4646250
[soft step  399] loss=0.4541  ce=0.4642899
Init CE: 1.31172776222229
[pass 1/12] pos  1 -> loss 1.1821862
[pass 1/12] pos  2 -> loss 0.9433925
[pass 1/12] pos  3 -> loss 0.8701335
[pass 1/12] pos  4 -> loss 0.7887989
[pass 2/12] pos  0 -> loss 0.6890621
[pass 3/12] pos  1 -> loss 0.6315801
[pass 4/12] no improv

In [16]:
y_ref_final = {s: greedy_generate_text(P_REF, s, **GEN_KW) for s in SUFFIXES}
for s in SUFFIXES:
    ref = y_ref_final[s]
    out = greedy_generate_ids(cand.prefix_ids, s, **GEN_KW)
    print("\nSUFFIX:", s)
    print("SAME?", out == ref)
    print("REF :", ref[:240].replace("\n","\\n"), "...")
    print("CAND:", out[:240].replace("\n","\\n"), "...")


SUFFIX: Write exactly two sentences explaining a simple process.
SAME? False
REF : Cats groom themselves by licking their fur, removing dirt and loose hairs.  This process helps keep their coat healthy and shiny. \n ...
CAND: Cats purr when they are happy or content.  They also purr when they are injured or sick, as a way to communicate their pain. \n ...

SUFFIX: Output valid JSON with keys "a" and "b", values as integers.
SAME? False
REF : ```json\n{\n  "a": 3,\n  "b": 17\n}\n``` ...
CAND: ```json\n{\n  "a": 42,\n  "b": 17\n}\n``` \n ...

SUFFIX: Ask one clarifying question before providing any answer.
SAME? False
REF : What kind of cat are you interested in learning about?  (e.g., domestic shorthair, Persian, Siamese, etc.) \n ...
CAND: What specific aspect of cats are you interested in learning about? \n ...

SUFFIX: If unsure, say you are not sure and explain why.
SAME? False
REF : Okay, I can talk about cats!  What would you like to know? 😸 \n\nFor example, I can tell you about: